<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/master/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code (Frontend and IPFS Connection)

In [1]:
#@title Config and Requirements
!pip install coolname gdown papermill json5
from coolname import generate_slug
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
IPFS_PEER = "ipfs.pollinations.ai"
ipfs_root = "/ipfs"
worker_root = "/content"
print(f"---Settings---\nIPFS_ROOT: {ipfs_root}\nIPFS_PEER: {IPFS_PEER}")

     |████████████████████████████████| 143kB 14.8MB/s 
     |████████████████████████████████| 122kB 42.2MB/s 
     |████████████████████████████████| 727kB 46.7MB/s 
     |████████████████████████████████| 747kB 45.5MB/s 
  Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5


---Settings---
IPFS_ROOT: /ipfs
IPFS_PEER: ipfs.pollinations.ai


In [2]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!mkdir -p $worker_root
!echo "Created IPFS resource" > $ipfs_root/output/log



rm: cannot remove '/ipfs': No such file or directory


In [3]:
#@title Install IPFS

%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout dev 
!git pull
!npm run install_backend
%cd -



/content
Cloning into 'pollinations'...
remote: Enumerating objects: 2266, done.
remote: Counting objects: 100% (658/658), done.
remote: Compressing objects: 100% (525/525), done.
remote: Total 2266 (delta 448), reused 256 (delta 129), pack-reused 1608
Receiving objects: 100% (2266/2266), 18.72 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (1491/1491), done.
/content/pollinations/app
Already up to date.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'
Already up to date.

> pollinations@0.4.5 install_backend /content/pollinations/app
> cp -v dist/pollinate.js /usr/local/bin/pollinate

'dist/pollinate.js' -> '/usr/local/bin/pollinate'
/content


In [4]:
!/content/pollinations/ipfs/ipfsd_install_colab.sh
!nohup  /content/pollinations/ipfs/ipfsd_forever.sh > $ipfs_root/output/log &
!sleep 5



--2021-06-12 06:51:44--  https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
Resolving dist.ipfs.io (dist.ipfs.io)... 209.94.78.1, 2602:fea2:3::1
Connecting to dist.ipfs.io (dist.ipfs.io)|209.94.78.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25904633 (25M) [application/gzip]
Saving to: ‘go-ipfs_v0.8.0_linux-amd64.tar.gz’

go-ipfs_v0.8.0_linu 100%[===================>]  24.70M  72.5MB/s    in 0.3s    

2021-06-12 06:51:45 (72.5 MB/s) - ‘go-ipfs_v0.8.0_linux-amd64.tar.gz’ saved [25904633/25904633]

go-ipfs/install.sh
go-ipfs/ipfs
go-ipfs/LICENSE
go-ipfs/LICENSE-APACHE
go-ipfs/LICENSE-MIT
go-ipfs/README.md
Moved ./ipfs to /usr/local/bin
/content
generating ED25519 keypair...done
peer identity: 12D3KooWJk1dSCYeGKBUj6PbDNnqB4gAa2EcwjBJDZ8qvy6U9ZMn
initializing IPFS node at /root/.ipfs
to get started, enter:

	ipfs cat /ipfs/QmQPeNsJPyVWPFDVHb77w8G42Fvo15z4bG2X8D2GhfbSXc/readme

nohup: redirecting stderr to stdout


In [5]:
from os.path import join


notebook_path = join(ipfs_root,"notebook.ipynb")



In [6]:
node_id=!ipfs id -f="<id>\n"
node_id=node_id[0]

print("---Node ID---:", node_id)

---Node ID---: 12D3KooWJk1dSCYeGKBUj6PbDNnqB4gAa2EcwjBJDZ8qvy6U9ZMn


In [7]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `https://pollinations.ai/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe)
  const iframe2 = document.createElement("iframe")
  iframe2.src = `http://localhost:3000/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe2)
'''))

!sleep 5


<IPython.core.display.Javascript object>

In [8]:
# !DEBUG=* pollinate -p {ipfs_root} --ipns --send --once

In [9]:
!DEBUG=* pollinate -p {ipfs_root} --ipns --receive --once

  ipfsConnector NodeID Promise { <pending> } +0ms
  ipfsWatch CLI options {
  path: '/ipfs',
  receive: true,
  send: false,
  once: true,
  ipns: true,
  nodeid: null
} +0ms
  ipfsWatch IPNS activated. subscring to CIDs +1ms
  ipfsConnector Localhost:5001 is reachable. Connecting... +8ms
  ipfsConnector Subscribing to pubsub events from 12D3KooWJk1dSCYeGKBUj6PbDNnqB4gAa2EcwjBJDZ8qvy6U9ZMn +67ms
  ipfsWatch remoteCID from pubsub QmVg8GQh7H6WKVaKaYyBm98AEVL9ARfe2zJ5nQH3p7Hxj4 +6m
  ipfsWatch Processing remote CID QmVg8GQh7H6WKVaKaYyBm98AEVL9ARfe2zJ5nQH3p7Hxj4 +0ms
  ipfsState Getting state for CID QmVg8GQh7H6WKVaKaYyBm98AEVL9ARfe2zJ5nQH3p7Hxj4 +0ms
  ipfsState:_getIPFSState(/) Getting state for dir root QmVg8GQh7H6WKVaKaYyBm98AEVL9ARfe2zJ5nQH3p7Hxj4 +0ms
  utils:ipfsls --- Calling  ipfsls with input QmVg8GQh7H6WKVaKaYyBm98AEVL9ARfe2zJ5nQH3p7Hxj4 +0ms
  utils:ipfsls --- In: QmVg8GQh7H6WKVaKaYyBm98AEVL9ARfe2zJ5nQH3p7Hxj4 +1ms
  utils:ipfsls --- Out: ipfsls : [
  {
    name: 'input',
    p

In [10]:
from glob import glob
from os.path import basename
from json5 import loads
parameters={}
for path in glob(f"{ipfs_root}/input/*"):
  key = basename(path)
  with open(path,'r') as f: 
    value = f.read()
    parameters[key] = loads(value)

print("New parameters", parameters)

New parameters {'text_not': 'disconnected, confusing, incoherent, cluttered, watermarks, text, writing', 'text_input': 'Vibrant painting of a synthesizer picnic on a sunny day in the garden of the gasworks with two large chimneys in the background. '}


In [11]:
!nohup pollinate -p {ipfs_root} --ipns --send &

nohup: appending output to 'nohup.out'


In [ ]:
#!pip install --upgrade pip

#!pip install --upgrade ipython
#import papermill as pm

from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

output_path = ipfs_root+"/output"
#parameters["text"] = "A black and white photo of Saeko with her new Elektron FM synthesizer"
def run(parameters):
  print("Running",parameters)
 
  parameters["output_path"] = output_path
  _params = [["-p", key, value] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]
  cmd = ["papermill", notebook_path, "/content/notebook_output.ipynb"] + params
  %run {notebook_path}
  return
  print("Running cmd"," ".join(cmd))
  p = Popen(
          cmd, 
          stdout = PIPE, 
          stderr = STDOUT, 
          shell = False, 
          cwd = worker_root
          )


  while True:
    line = p.stdout.readline()#
    if not line: break
    line_text = line.decode("utf-8")

    stdout.write(line)
    with open(f"{ipfs_root}/output/log","a") as log_out:
      if not line_text.startswith("unhandled"):
        log_out.write(line_text)
        log_out.flush()
      
   
        
while True:
  run(parameters)
  !sleep 100

Running {'text_not': 'disconnected, confusing, incoherent, cluttered, watermarks, text, writing', 'text_input': 'Vibrant painting of a synthesizer picnic on a sunny day in the garden of the gasworks with two large chimneys in the background. '}
  Created wheel for slugify: filename=slugify-0.0.1-cp37-none-any.whl size=1909 sha256=b7e2371afc1c7b8eb062ae62b32001a321b151d5d1f8935c385ec1e3c50420a9
  Stored in directory: /root/.cache/pip/wheels/f9/68/7b/b797d07992b6c7ef1666a59cc79f45fb37aab4344efa6cc3e1
Successfully built slugify
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-1a5d71b8-83e9-cf91-1aeb-3498890f200b)
     |████████████████████████████████| 286kB 8.6MB/s 


/usr/local/lib/python3.7/dist-packages/kornia/augmentation/augmentation.py:1833: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,


     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=81928861154de17cc07f83935d44704e5ff2c221312ec9185d7632bf55a1caad
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
Successfully built ftfy
Cloning into 'CLIP'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), done.


100%|███████████████████████████████████████| 354M/354M [00:05<00:00, 60.8MiB/s]
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Uninstalling torchtext-0.9.1:
  Successfully uninstalled torchtext-0.9.1
/content
Cloning into 'taming-transformers'...
remote: Enumerating objects: 685, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 685 (delta 9), reused 19 (delta 6), pack-reused 658
Receiving objects: 100% (685/685), 116.29 MiB | 36.07 MiB/s, done.
Resolving deltas: 100% (144/144), done.
/content/taming-transformers
--2021-06-12 06:58:52--  https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/03400bb1-d44d-4c33-ba96-ad58bab5a0ef/last.ckpt [following]
--2021-06-12 06:58:53--  https://heibox.uni-heidelberg.de/seafhttp/files/03400bb1-d44d-4c33-ba96-ad58bab5a0ef/l